В этом задании вам предлагается начать разбираться с задачей анализа тональности отзывов на примере сентимент-анализа отзывов на фильмы.

Мы будем использовать стандартный датасет из nltk, уже возникавший в одном из примеров в предыдущих курсах. Для того, чтобы импортировать необходимый модуль, напишите:

In [96]:
from nltk.corpus import movie_reviews
import numpy as np


Чтобы получить id-шники негативных и позитивных отзывов:

In [3]:
negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')

print(negids[:5])

['neg/cv000_29416.txt', 'neg/cv001_19502.txt', 'neg/cv002_17424.txt', 'neg/cv003_12683.txt', 'neg/cv004_12641.txt']


Чтобы получить список негативных отзывов:

In [4]:
negfeats = [movie_reviews.words(fileids=[f]) for f in negids]

In [14]:
negfeats[0]

['plot', ':', 'two', 'teen', 'couples', 'go', 'to', ...]

Создайте список из текстов всех имеющихся отзывов, а также список с классами, которые будет использовать ваш классификатор - 0 для негативных отзывов и 1 для позитивных.

In [16]:
posfeats = [movie_reviews.words(fileids=[f]) for f in posids]

In [29]:
movie_reviews.words(fileids=posids[0])

['films', 'adapted', 'from', 'comic', 'books', 'have', ...]

In [67]:
texts = negfeats + posfeats
labels = [0] * len(negfeats) + [1] * len(posfeats)

In [20]:
labels[-1]

1

Подсчитайте количество отзывов в выборке

In [33]:
len(texts)

2000

Подсчитайте долю класса 1 в выборке

In [34]:
len(posfeats)/len(texts)*100

50.0

Импортируйте CountVectorizer из sklearn.feature_extraction.text. Попробуйте использовать его с настройками по умолчанию для того, чтобы получить признаковое представление каждого текста. Скорее всего, попытка не увенчается успехом. Разберитесь, в чем причина, и добейтесь того, чтобы метод fit_transform у CountVectorizer успешно отрабатывал. Подсчитайте количество признаков в CountVectorizer. Никакой предварительной обработки текста (удаление стоп-слов, нормализация слов) на этом шаге делать не надо, в качестве признаков должны использоваться частоты слов.

In [35]:
from sklearn.feature_extraction.text import CountVectorizer

In [36]:
vectorizer = CountVectorizer()

In [37]:
vectorizer.fit_transform(texts)

AttributeError: 'StreamBackedCorpusView' object has no attribute 'lower'

**приведем каждый текст в строку**

In [68]:
texts[0]

['plot', ':', 'two', 'teen', 'couples', 'go', 'to', ...]

In [70]:
new_texts = [" ".join(texts[i]) for i in range(len(texts))]

In [73]:
X = vectorizer.fit_transform(new_texts)

Соберите pipeline из CountVectorizer и LogisticRegression c настройками по-умолчанию и с помощью cross_val_score (также со стандартными настройками) оцените получаемое "из коробки" качество по accuracy

In [78]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
import warnings
warnings.filterwarnings('ignore')

In [79]:
pipeline = Pipeline([('count_vect', CountVectorizer()), ('log_reg', LogisticRegression(random_state=122))])

In [80]:
cross_val_score(pipeline, new_texts, labels, scoring='accuracy', cv=10).mean()

0.8445

In [82]:
cross_val_score(LogisticRegression(random_state=122), X, labels,scoring='accuracy', cv=10).mean()

0.8445

Аналогично accuracy, оцените качество по ROC AUC

In [83]:
cross_val_score(pipeline, new_texts, labels, scoring='roc_auc', cv=10).mean()

0.9189399999999999

Обучите логистическую регрессию на всей доступной вам выборке и выведите 5 наиболее важных для модели признаков (подумайте, какие именно признаки стоит считать такими). Вам могут пригодиться метод get_feature_names() или поле vocabulary_ у класса CountVectorizer

In [85]:
log = LogisticRegression()

In [87]:
log.fit(X,labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [140]:
result = list(zip(abs(log.coef_[0]), vectorizer.get_feature_names()))

In [144]:
result[0]

(0.011632980025645225, '00')

In [147]:
sorted(result, reverse=True)[:5]

[(0.8134354406678922, 'bad'),
 (0.640742793247156, 'unfortunately'),
 (0.6025749258275003, 'worst'),
 (0.5658206766485677, 'fun'),
 (0.5201987262700075, 'nothing')]

In [165]:
result1 = list(zip(abs(log.coef_[0]), sorted(vectorizer.vocabulary_)))

In [167]:
sorted(result1, reverse=True)[:5]

[(0.8134354406678922, 'bad'),
 (0.640742793247156, 'unfortunately'),
 (0.6025749258275003, 'worst'),
 (0.5658206766485677, 'fun'),
 (0.5201987262700075, 'nothing')]